# Training of a simple encoder transformer
- trained with a small balanced dataset 

In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)
    
MODEL_V = "04_simple_transformer"

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='cpu')

In [3]:
import torch
print(torch.cuda.is_available())  

False


## Splitting the data

In [66]:
from Dataset import Dataset

data_loader = Dataset(
    data_path="imputed_sofa.parquet",
    save_path="dataset_tensors.pth",
    method="oversample",
    balance=True,
    minority_ratio=0.5,
    target_column="SepsisLabel"
)
# as tensors
X_train, X_test, y_train, y_test = data_loader.get_train_test_tensors(size='small', train_size=0.1)
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)


Balanced training set balance:
SepsisLabel
0    1219428
1     609714
Name: count, dtype: int64
Reduced balanced training set balance:
SepsisLabel
0    121943
1     60971
Name: count, dtype: int64
Total records in reduced training set: 182914


## Transformer architecture

In [67]:
from architectures import TransformerClassifier


in_dim = X_train.shape[1]
model = TransformerClassifier(input_dim=in_dim)

In [68]:
from torch.utils.data import DataLoader, TensorDataset


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 512 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Training loop

In [69]:
negative_count = (y_train == 0).sum().item()  # Count negatives (class 0)
positive_count = (y_train == 1).sum().item()  # Count positives (class 1)

if positive_count == 0:
    raise ValueError("No positive samples found in training set.")

print(negative_count / positive_count)

2.0000164012399337


In [70]:
from tqdm import tqdm
from torchmetrics import Accuracy, Precision, Recall
from torchvision.ops import sigmoid_focal_loss
from torch import nn


model = TransformerClassifier(input_dim=in_dim).to(device)
pos_weight = torch.tensor([2], dtype=torch.float32).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001) 

epochs = 100
epoch_counter = []
loss_counter = []
acc_counter = []

t_accuracy = Accuracy(task='binary').to(device)
t_precision = Precision(task='binary').to(device)
t_recall = Recall(task='binary').to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss, epoch_acc, epoch_prec, epoch_rec = 0, 0, 0, 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # forward pass
        y_logits = model(X_batch)
        # print("Logits sample:", y_logits[:5].detach().cpu().numpy())  # Should have positive values too
        y_probs = torch.sigmoid(y_logits)
        # because the dataset is too imbalance
        y_preds = torch.round(y_probs)

        # loss function
        loss = loss_fn(y_logits, y_batch.unsqueeze(1).float())
        
        acc = t_accuracy(y_preds, y_batch.unsqueeze(1).float())
        prec = t_precision(y_preds, y_batch.unsqueeze(1).float())
        rec = t_recall(y_preds, y_batch.unsqueeze(1).float())
        
        # zero grad
        optimizer.zero_grad()
        # backpropagation
        loss.backward()
        # optimizer
        optimizer.step()

        # update epoch metrics
        epoch_loss += loss.item()
        epoch_acc += acc
        epoch_prec += prec
        epoch_rec += rec
        
        progress_bar.set_postfix({"Loss": loss.item(), "Acc": acc.item(), "Prec": prec.item(), "Rec": rec.item()})

    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)
    epoch_prec /= len(train_loader)
    epoch_rec /= len(train_loader)
    acc_counter.append(epoch_acc)
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.5f} | Accuracy: {epoch_acc:.2f}% | Precision: {epoch_prec:.2f}% | Recall: {epoch_rec:.2f}%")

Epoch 1/100:   0%|          | 0/358 [01:45<?, ?it/s, Loss=0.968, Acc=0.554, Prec=0.41, Rec=0.314]  

Epoch 1/100 | Loss: 0.92927 | Accuracy: 0.50% | Precision: 0.33% | Recall: 0.50%


Epoch 2/100 | Loss: 0.92502 | Accuracy: 0.49% | Precision: 0.32% | Recall: 0.52%


Epoch 3/100:   0%|          | 0/358 [01:53<?, ?it/s, Loss=0.959, Acc=0.431, Prec=0.391, Rec=0.86]   

Epoch 3/100 | Loss: 0.92464 | Accuracy: 0.50% | Precision: 0.32% | Recall: 0.51%


...

Epoch 97/100 | Loss: 0.92417 | Accuracy: 0.50% | Precision: 0.30% | Recall: 0.49%


Epoch 98/100 | Loss: 0.92424 | Accuracy: 0.56% | Precision: 0.34% | Recall: 0.33%


Epoch 99/100:   0%|          | 0/358 [02:08<?, ?it/s, Loss=0.895, Acc=0.708, Prec=0, Rec=0]          

Epoch 99/100 | Loss: 0.92416 | Accuracy: 0.48% | Precision: 0.29% | Recall: 0.55%


Epoch 100/100 | Loss: 0.92416 | Accuracy: 0.51% | Precision: 0.25% | Recall: 0.47%


## Save model and predictions

In [11]:
from pathlib import Path

# save the model
model_path = Path('./saved')
model_path.mkdir(exist_ok=True)
model_file = model_path / f"{MODEL_V}.pth"

torch.save(model.state_dict(), model_file)

In [12]:
model = TransformerClassifier(input_dim=in_dim, num_heads=n_heads)
model.load_state_dict(torch.load(f"./saved/{MODEL_V}.pth"))

/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/venv-sepsis/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>